In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np
import seaborn as sns
import re

In [2]:
train = pd.read_csv(r'C:\Users\rajat\Downloads\titanic\train.csv')
test = pd.read_csv(r'C:\Users\rajat\Downloads\titanic\test.csv')
dataset = [train,test]


In [3]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
data = [train,test]
start = ', '
end = '. '
desig=[]
for dataset in data:
    desig=[]
    for s in dataset.Name:
        desig.append(s[s.find(start)+len(start):s.find(end)])
    desig=np.array(desig)
    dataset['Title']=desig
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [6]:
data = pd.concat([train.drop(axis=1,columns=['Survived']),test])
data_median=data.groupby(['Title','Pclass','Sex']).agg({'Age':'median'})
data_median = data_median.reset_index()[['Sex', 'Pclass', 'Title', 'Age']]
data_median

,Sex,Pclass,Title,Age
0,male,1,Capt,70.0
1,male,1,Col,54.5
2,male,1,Don,40.0
3,female,1,Dona,39.0
4,female,1,Dr,49.0
5,male,1,Dr,47.0
6,male,2,Dr,38.5
7,male,1,Jonkheer,38.0
8,female,1,Lady,48.0
9,male,1,Major,48.5


In [7]:
def fill_age(row):
    condition = (
        (data_median['Sex'] == row['Sex']) & 
        (data_median['Title'] == row['Title']) & 
        (data_median['Pclass'] == row['Pclass'])
    ) 
    return data_median[condition]['Age'].values[0]


def process_age():
    global data

    data['Age'] = data.apply(lambda row: fill_age(row) if np.isnan(row['Age']) else row['Age'], axis=1)
    return data

data = process_age()
data.loc[data.index[88], 'Age'] = "18.0"


In [8]:
data.drop(['Name'],axis=1,inplace=True)
t_dummy = pd.get_dummies(data['Title'],prefix='Title')
data = pd.concat([data,t_dummy],axis=1)
data.drop(['Title'],axis=1,inplace=True)

In [9]:
data_fare=data.groupby(['Sex']).agg({'Fare':'mean'})
data_fare = data_fare.reset_index()
data_fare

,Sex,Fare
0,female,46.198097
1,male,26.154601


In [10]:
mapping= data['Sex'].map(data.groupby(['Sex'])['Fare'].mean())
data.Fare.fillna(value=mapping)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PassengerId         1309 non-null   int64  
 1   Pclass              1309 non-null   int64  
 2   Sex                 1309 non-null   object 
 3   Age                 1309 non-null   object 
 4   SibSp               1309 non-null   int64  
 5   Parch               1309 non-null   int64  
 6   Ticket              1309 non-null   object 
 7   Fare                1308 non-null   float64
 8   Cabin               295 non-null    object 
 9   Embarked            1307 non-null   object 
 10  Title_Capt          1309 non-null   uint8  
 11  Title_Col           1309 non-null   uint8  
 12  Title_Don           1309 non-null   uint8  
 13  Title_Dona          1309 non-null   uint8  
 14  Title_Dr            1309 non-null   uint8  
 15  Title_Jonkheer      1309 non-null   uint8  
 16  Title_L

In [11]:
data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,1,3,male,22,1,0,A/5 21171,7.2500,NaN,S,...,0,0,0,0,1,0,0,0,0,0
1,2,1,female,38,1,0,PC 17599,71.2833,C85,C,...,0,0,0,0,0,1,0,0,0,0
2,3,3,female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,...,0,1,0,0,0,0,0,0,0,0
3,4,1,female,35,1,0,113803,53.1000,C123,S,...,0,0,0,0,0,1,0,0,0,0
4,5,3,male,35,0,0,373450,8.0500,NaN,S,...,0,0,0,0,1,0,0,0,0,0


In [12]:
data.Embarked.fillna('S',inplace=True)

In [13]:
data.Fare.fillna('26.15',inplace=True)

In [14]:
emb_dummy = pd.get_dummies(data['Embarked'],prefix='Embarked')
data = pd.concat([data,emb_dummy],axis=1)
data.drop(['Embarked'],axis=1,inplace=True)


In [15]:
data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title_Capt,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Embarked_C,Embarked_Q,Embarked_S
0,1,3,male,22,1,0,A/5 21171,7.25,NaN,0,...,0,1,0,0,0,0,0,0,0,1
1,2,1,female,38,1,0,PC 17599,71.2833,C85,0,...,0,0,1,0,0,0,0,1,0,0
2,3,3,female,26,0,0,STON/O2. 3101282,7.925,NaN,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,female,35,1,0,113803,53.1,C123,0,...,0,0,1,0,0,0,0,0,0,1
4,5,3,male,35,0,0,373450,8.05,NaN,0,...,0,1,0,0,0,0,0,0,0,1


In [16]:
train.groupby(['Cabin']).agg({'Survived':'mean'})

,Survived
Cabin,
A10,0.0
A14,0.0
A16,1.0
A19,0.0
A20,1.0
...,...
F33,1.0
F38,0.0
F4,1.0


In [17]:
map_= train.groupby(['Cabin']).Survived.mean().astype(int)
data['Cabin'] = data['Cabin'].map(map_)
data['Cabin'].fillna(value=2,inplace=True)
data    

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title_Capt,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Embarked_C,Embarked_Q,Embarked_S
0,1,3,male,22,1,0,A/5 21171,7.25,2.0,0,...,0,1,0,0,0,0,0,0,0,1
1,2,1,female,38,1,0,PC 17599,71.2833,1.0,0,...,0,0,1,0,0,0,0,1,0,0
2,3,3,female,26,0,0,STON/O2. 3101282,7.925,2.0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,female,35,1,0,113803,53.1,0.0,0,...,0,0,1,0,0,0,0,0,0,1
4,5,3,male,35,0,0,373450,8.05,2.0,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,26,0,0,A.5. 3236,8.05,2.0,0,...,0,1,0,0,0,0,0,0,0,1
414,1306,1,female,39,0,0,PC 17758,108.9,2.0,0,...,0,0,0,0,0,0,0,1,0,0
415,1307,3,male,38.5,0,0,SOTON/O.Q. 3101262,7.25,2.0,0,...,0,1,0,0,0,0,0,0,0,1
416,1308,3,male,26,0,0,359309,8.05,2.0,0,...,0,1,0,0,0,0,0,0,0,1


In [18]:
data.isnull().sum()

PassengerId           0
Pclass                0
Sex                   0
Age                   0
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Cabin                 0
Title_Capt            0
Title_Col             0
Title_Don             0
Title_Dona            0
Title_Dr              0
Title_Jonkheer        0
Title_Lady            0
Title_Major           0
Title_Master          0
Title_Miss            0
Title_Mlle            0
Title_Mme             0
Title_Mr              0
Title_Mrs             0
Title_Ms              0
Title_Rev             0
Title_Sir             0
Title_the Countess    0
Embarked_C            0
Embarked_Q            0
Embarked_S            0
dtype: int64

In [19]:
pclass = pd.get_dummies(data['Pclass'],prefix='Pclass')
data = pd.concat([data,pclass],axis=1)
data.drop(['Pclass'],inplace=True,axis=1)
data

,PassengerId,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title_Capt,Title_Col,...,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,1,male,22,1,0,A/5 21171,7.25,2.0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,2,female,38,1,0,PC 17599,71.2833,1.0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,3,female,26,0,0,STON/O2. 3101282,7.925,2.0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,4,female,35,1,0,113803,53.1,0.0,0,0,...,0,0,0,0,0,0,1,1,0,0
4,5,male,35,0,0,373450,8.05,2.0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,male,26,0,0,A.5. 3236,8.05,2.0,0,0,...,0,0,0,0,0,0,1,0,0,1
414,1306,female,39,0,0,PC 17758,108.9,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
415,1307,male,38.5,0,0,SOTON/O.Q. 3101262,7.25,2.0,0,0,...,0,0,0,0,0,0,1,0,0,1
416,1308,male,26,0,0,359309,8.05,2.0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [20]:
mapping = data.groupby('Ticket').Ticket.count()
data['Group'] = data['Ticket'].map(mapping)-1
data

,PassengerId,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title_Capt,Title_Col,...,Title_Rev,Title_Sir,Title_the Countess,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Group
0,1,male,22,1,0,A/5 21171,7.25,2.0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,2,female,38,1,0,PC 17599,71.2833,1.0,0,0,...,0,0,0,1,0,0,1,0,0,1
2,3,female,26,0,0,STON/O2. 3101282,7.925,2.0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,4,female,35,1,0,113803,53.1,0.0,0,0,...,0,0,0,0,0,1,1,0,0,1
4,5,male,35,0,0,373450,8.05,2.0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,male,26,0,0,A.5. 3236,8.05,2.0,0,0,...,0,0,0,0,0,1,0,0,1,0
414,1306,female,39,0,0,PC 17758,108.9,2.0,0,0,...,0,0,0,1,0,0,1,0,0,2
415,1307,male,38.5,0,0,SOTON/O.Q. 3101262,7.25,2.0,0,0,...,0,0,0,0,0,1,0,0,1,0
416,1308,male,26,0,0,359309,8.05,2.0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [21]:
data["Alone"] = data.Parch+data.SibSp+data.Group
data['Alone']= data['Alone'].apply(lambda x: 1 if x>0 else 0)
data


,PassengerId,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title_Capt,Title_Col,...,Title_Sir,Title_the Countess,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Group,Alone
0,1,male,22,1,0,A/5 21171,7.25,2.0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,2,female,38,1,0,PC 17599,71.2833,1.0,0,0,...,0,0,1,0,0,1,0,0,1,1
2,3,female,26,0,0,STON/O2. 3101282,7.925,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,4,female,35,1,0,113803,53.1,0.0,0,0,...,0,0,0,0,1,1,0,0,1,1
4,5,male,35,0,0,373450,8.05,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,male,26,0,0,A.5. 3236,8.05,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
414,1306,female,39,0,0,PC 17758,108.9,2.0,0,0,...,0,0,1,0,0,1,0,0,2,1
415,1307,male,38.5,0,0,SOTON/O.Q. 3101262,7.25,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
416,1308,male,26,0,0,359309,8.05,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [22]:
data['Sex'] = data['Sex'].map({'male':1, 'female':0})
data

,PassengerId,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title_Capt,Title_Col,...,Title_Sir,Title_the Countess,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Group,Alone
0,1,1,22,1,0,A/5 21171,7.25,2.0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,2,0,38,1,0,PC 17599,71.2833,1.0,0,0,...,0,0,1,0,0,1,0,0,1,1
2,3,0,26,0,0,STON/O2. 3101282,7.925,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,4,0,35,1,0,113803,53.1,0.0,0,0,...,0,0,0,0,1,1,0,0,1,1
4,5,1,35,0,0,373450,8.05,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,1,26,0,0,A.5. 3236,8.05,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
414,1306,0,39,0,0,PC 17758,108.9,2.0,0,0,...,0,0,1,0,0,1,0,0,2,1
415,1307,1,38.5,0,0,SOTON/O.Q. 3101262,7.25,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0
416,1308,1,26,0,0,359309,8.05,2.0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [181]:
def train_test_targets():
    global data
    
    targets = pd.read_csv(r'C:\Users\rajat\Downloads\titanic\train.csv', usecols=['Survived'])['Survived'].values
    train = data.iloc[:891]
    test = data.iloc[891:]
    
    return train, test, targets

train, test, targets = train_test_targets()


In [182]:
train.drop(columns=['PassengerId','Ticket'],inplace=True)
test.drop(columns=['PassengerId','Ticket'],inplace=True)

In [183]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, targets, test_size=0.33, random_state=42)

In [184]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test = sc.fit_transform(test)

In [185]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [186]:
from tensorflow import keras
from keras.constraints import max_norm 

In [187]:
classifier = keras.models.Sequential()

In [188]:
classifier.add(keras.layers.Flatten())

classifier.add(keras.layers.Dense(5, kernel_constraint=max_norm(3),activation = 'relu'))
#Hidden layer

classifier.add(keras.layers.Dense(2,kernel_constraint=max_norm(3),activation = 'relu'))
classifier.add(keras.layers.Dense(1 ,kernel_constraint=max_norm(3),activation = 'sigmoid'))

In [189]:
classifier.compile(loss="binary_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [190]:
classifier.fit(X_train, y_train,batch_size=10,epochs = 100)


Epoch 1/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.6275
Epoch 2/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6650 - accuracy: 0.6275
Epoch 3/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6571 - accuracy: 0.6275
Epoch 4/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6500 - accuracy: 0.6275
Epoch 5/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6424 - accuracy: 0.6275
Epoch 6/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6341 - accuracy: 0.6275
Epoch 7/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6253 - accuracy: 0.6275
Epoch 8/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6172 - accuracy: 0.6275
Epoch 9/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6096 - accuracy: 0.6275
Epoch 10/100
60/60 [==============================] - 0s 2ms/step - loss: 0.6023 - accuracy: 0.6275
Epoch 11/

60/60 [==============================] - 0s 2ms/step - loss: 0.3853 - accuracy: 0.8490
Epoch 84/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3847 - accuracy: 0.8523
Epoch 85/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3839 - accuracy: 0.8473
Epoch 86/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3834 - accuracy: 0.8507
Epoch 87/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3825 - accuracy: 0.8473
Epoch 88/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3818 - accuracy: 0.8507
Epoch 89/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3810 - accuracy: 0.8473
Epoch 90/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3800 - accuracy: 0.8473
Epoch 91/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3792 - accuracy: 0.8490
Epoch 92/100
60/60 [==============================] - 0s 2ms/step - loss: 0.3783 - accuracy: 0.8490
Epoch 93/100


In [191]:
y_test = np.asarray(y_test).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

In [192]:
classifier.evaluate(X_test,y_test)

10/10 [==============================] - 0s 2ms/step - loss: 1.1076 - accuracy: 0.7627


[1.1075541973114014, 0.7627118825912476]

In [202]:
pre = classifier.predict(test)

In [218]:
predictions = pd.DataFrame(pre)
predictions

,0
0,0.075974
1,0.330373
2,0.145711
3,0.086810
4,0.579997
...,...
413,0.087384
414,0.841568
415,0.069934
416,0.087384


In [220]:
series = []
ax = pd.read_csv(r'C:\Users\rajat\Downloads\titanic\test.csv')
predictions['PassengerId'] = ax['PassengerId']
for val in predictions[0]:
    if val >= 0.5:
        series.append(1)
    else:
        series.append(0)
predictions['Survived'] = series
predictions

,0,PassengerId,Survived
0,0.075974,892,0
1,0.330373,893,0
2,0.145711,894,0
3,0.086810,895,0
4,0.579997,896,1
...,...,...,...
413,0.087384,1305,0
414,0.841568,1306,1
415,0.069934,1307,0
416,0.087384,1308,0


In [222]:


predictions.to_csv("rajat.csv" , index=False)